# LisPy Interpreter 
This notebook implements a subset of the scheme dialect of Lisp using Python

In [1]:
Symbol = str
List   = list
Number = (int, float)

def parse( program ):
    # Read a Scheme expression from a string
    return read_from_tokens(tokenize(program))

def tokenize( string ):
    # Convert a string of characters into a list of tokens
    return string.replace("("," ( ").replace(")"," ) ").split()

def read_from_tokens( tokens ):
    # Read an expression from a sequence of tokens
    if not len(tokens): raise SyntaxError('unexpected EOF while reading')
    token = tokens.pop(0)
    if   "(" == token:
        line = []
        while tokens[0] != ")": line.append(read_from_tokens(tokens))
        tokens.pop(0)
        return line
    elif ")" == token: raise SyntaxError("Unexpected")
    else: return atom(token)
    
def atom( token ):
    # Numbers become numbers, every other token is a symbol
    try:                   return int(token)
    except ValueError:
        try:               return float(token)
        except ValueError: return Symbol(token)


An environment is a mapping of procedures and variable names to their values.

In [2]:
import math
import operator as op

Env = dict     # An environment is a mapping of {variable: value}

def standard_env():
    # An environment with some Scheme standard procedures
    env = Env()
    env.update(vars(math))
    env.update({
        "+":op.add, "-":op.sub, "*":op.mul, "/":op.div,
        ">":op.gt,  "<":op.lt,  ">=":op.ge, "<=":op.le, "=":op.eq,
        "abs":         abs,
        "append":      op.add,
        "apply":       apply,
        "begin":       lambda *x: x[-1],
        "car":         lambda x: x[0],
        "cdr":         lambda x: x[1:],
        "cons":        lambda x,y: [x] + y,
        "eq?":         op.is_,
        "equal?":      op.eq,
        "length":      len,
        "list":        lambda x: list(x),
        "list?":       lambda x: isinstance(x, List),
        "map":         map,
        "max":         max,
        "min":         min,
        "not":         op.not_,
        "null?":       lambda x: x == [],
        "number?":     lambda x: isinstance(x, Number),
        "procedure?":  callable,
        "round":       round,
        "symbol?":     lambda x: isinstance(x, Symbol),
    })
    return env
        
global_env = standard_env()

Eval follows each line of the code to follow procedures and augment environmenet with user defined variables.

In [3]:
def eval(x, env=global_env):
    # Evaluate an expression in an environment
    if isinstance(x, Symbol):       # variable reference
        return env[x]
    elif not isinstance(x, List):   # constant literal
        return x
    elif x[0] == "if":              # conditional
        (_, test, conseq, alt) = x
        exp = (conseq if eval(test, env) else alt)
        return eval(exp, env)
    elif x[0] == "define":          # definition
        (_, var, exp) = x
        env[var] = eval(exp, env)
    elif x[0] == "print":           # print 
        print eval(x[1], env)
    else:                           # procedure call
        proc = eval(x[0], env)
        args = [eval(arg, env) for arg in x[1:]]
        return proc(*args)
    
def run(script): return eval(parse(script))

Run the script. 

In [4]:
program = """

(begin 
  (define r 10) 
  (define area (* pi (* r r))) 
  (print area)
)

"""

run(program)

314.159265359


## Interpreter 

Let's also implement an interactive read-eval-print loop: a way for a programmer to enter an expression, and see it immediately read, evaluated and printed.

In [5]:
def repl(prompt='LisPy> '):
    # A prompt-read-eval-print loop.
    while True:
        val = eval(parse("(%s)" % raw_input(prompt)))
        if val is not None: print(schemestr(val))

def schemestr(exp):
    # Convert a Python object back into a Scheme-readable string.
    if isinstance(exp, List):  return '(' + ' '.join(map(schemestr, exp)) + ')' 
    else:                      return str(exp)

In [ ]:
repl()

LisPy> define r 10
LisPy> define l 4
LisPy> define vol (* pi (* r (* r l )))
LisPy> print vol
1256.63706144
